
---
# Load needed libraries and functions

In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
%pprint

notebook_path = get_notebook_path()
print(notebook_path)

s = Storage()
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
hunting_df = s.load_object('hunting_df')
basic_quals_dict = s.load_object('basic_quals_dict')
dir()

Pretty printing has been turned OFF
D:\Documents\Repositories\notebooks\Miscellaneous\ipynb\Job Hunting Data Exploration.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']


['Config', 'CountVectorizer', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', 'TfidfTransformer', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'basic_quals_dict', 'check_4_doubles', 'check_for_typos', 'conjunctify_list', 'copyfile', 'csv', 'encoding', 'exit', 'get_classifier', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_module_version', 'get_notebook_path', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'hunting_df', 'ipykernel', 'json', 'jupyter_config_dir', 'notebook_path', 'notebookapp', 'np', 'os', 'pd', 'pickle', 'preprocess_data', 'print_all_files_ending_starting_with', 'print_all_files_ending_with', 'print_all_files_starting_with', 'quit', 're', 'remove_empty_folders', 's', 'similar', '


---
# Needed extra functions

In [2]:

def print_loc_computation(row_index, quals_list, basic_quals_dict):
    print()
    numerator_str_list = []
    for qual_str in quals_list:
        if qual_str in basic_quals_dict:
            numerator_str_list.append(str(basic_quals_dict[qual_str]))
        else:
            numerator_str_list.append('000')
    numerator_str = '+'.join(numerator_str_list)
    print("hunting_df.loc[{}, 'percent_fit'] = ({})/{}".format(row_index, numerator_str, len(quals_list)))

In [3]:

def get_predictions_and_counts(prediction_list, quals_list):
    qual_count = 0
    prediction_str = ''
    for pred_array, qual_str in zip(prediction_list, quals_list):
        prediction = pred_array[1]
        prediction_str += '\n{} {}'.format(prediction, qual_str)
        if prediction > 0.5:
            qual_count += 1
    
    return prediction_str, qual_count

In [4]:

def get_quals_str(prediction_list, quals_list, basic_quals_dict):
    qual_count = 0
    quals_str = ''
    for pred_array, (i, qual_str) in zip(prediction_list, enumerate(quals_list)):
        if qual_str in basic_quals_dict:
            formatted_str = '\nquals_list[{}] = "{}" ({})'
        else:
            formatted_str = '\n*quals_list[{}] = "{}" ({})'
        prediction = pred_array[1]
        quals_str += formatted_str.format(i, qual_str, prediction)
        if prediction > 0.5:
            qual_count += 1
    
    return quals_str, qual_count

In [5]:

def print_fit_job(row_index, row_series, basic_quals_dict):
    job_fitness = 0.0
    job_description = row_series['Job Description']
    quals_list = get_quals_list(job_description)
    if len(quals_list):
        prediction_list = list(predict_percent_fit(quals_list))
        #prediction_str, qual_count = get_predictions_and_counts(prediction_list, quals_list)
        quals_str, qual_count = get_quals_str(prediction_list, quals_list, basic_quals_dict)
        job_fitness = qual_count/len(prediction_list)
        if job_fitness > 0.8:
            print('Basic Qualifications:{}'.format(quals_str))
            #print(prediction_str)
            print(job_fitness)
            print_loc_computation(row_index, quals_list, basic_quals_dict)
    
    return quals_list, job_fitness

In [6]:

def print_job_description(req_id):
    match_series = (hunting_df['Job Requisition ID'] == req_id)
    job_description = hunting_df[match_series]['Job Description'].tolist()[0]
    print(get_quals_list(job_description))
    print(job_description)

In [7]:

scanner_regex = re.compile(r'\b[1-9a-zA-Z][0-9a-zA-Z]*( *[#\+]{1,2}|\b)')
def regex_tokenizer(corpus):
    
    return [match.group() for match in re.finditer(scanner_regex, corpus)]

In [8]:

import string

printable_regex = re.compile('[^{}]+'.format(string.printable))
def un_msword_ify(x):
    msword_str = str(x)
    msword_str = printable_regex.sub(r' ', msword_str).strip()
    msword_str = re.sub(r'[^\x00-\x7f]+', r' ', msword_str).strip()
    msword_str = re.sub(r' +', ' ', msword_str)
    msword_str = re.sub(r'::', ':', msword_str)
    msword_str = re.sub(r':$', '', msword_str)
    msword_str = re.sub(r'^-', '', msword_str)
    
    return msword_str

In [9]:

a_list = ['Additional Qualifications?', 'Nice If You Have', 'Nice if you have', 'Nice if You Have',
          'Additional Preferred Qualifications', 'Nice if you Have', 'Additional qualifications', 'Nice to Have']
a_str = '({}):?'.format('|'.join(a_list))
def get_quals_list(job_description):
    job_description = un_msword_ify(job_description)
    basic_quals = ''
    quals_list = []
    items_list = re.split('(Key Role|The Challenge):', job_description, 0)
    if len(items_list) > 1:
        job_description = items_list[-1].strip()
    items_list = re.split('[\r\n]+(Basic Qualifications?|You Have|You have):?', job_description, 0)
    if len(items_list) > 1:
        job_description = items_list[-1].strip()
    items_list = re.split(a_str, job_description, 0)
    if len(items_list) > 1:
        basic_quals = items_list[0].strip()
    else:
        items_list = re.split('(Clearance|Build Your Career):', job_description, 0)
        basic_quals = items_list[0].strip()
    if basic_quals != '':
        quals_list = [un_msword_ify(q) for q in re.split('[\r\n]+', basic_quals, 0)]
        quals_list = [x for x in quals_list if x != '']
    
    return quals_list


---
# Data Exploration

In [10]:

req_id = 'R0081541'
match_series = (hunting_df['Job Requisition ID'] == req_id)
print(hunting_df[match_series].shape[0])
percent_fit = hunting_df.loc[match_series, 'percent_fit'].tolist()[0]
percent_fit

1


0.4

In [11]:

print_job_description('R0081541')

['Big Data', 'Big Data Management', 'Cloud', 'Containerization', 'Data Architecture', 'Database Access', 'Database Design', 'Database Management', 'Deep Learning', 'Domain Knowledge', 'Information Management', 'Machine Learning', 'Metadata Repository', 'Organizational Understanding', 'Peer-Reviewed Publications', 'Programming', 'Prototyping', 'Query Tools', 'Repository Management', 'Requirements Analysis', 'Statistics', 'Technical Troubleshooting', 'Technological Innovation', 'Technology Research', 'Thought Leadership']
Big Data

Big Data Management

Cloud

Containerization

Data Architecture

Database Access

Database Design

Database Management

Deep Learning

Domain Knowledge

Information Management

Machine Learning

Metadata Repository

Organizational Understanding

Peer-Reviewed Publications

Programming

Prototyping

Query Tools

Repository Management

Requirements Analysis

Statistics

Technical Troubleshooting

Technological Innovation

Technology Research

Thought Leadership


In [12]:

hunting_df[match_series].T

,5085
Hiring Manager,"Data Scientist, Senior"
Management Level,"USA, MA, Boston (150 S Huntington Ave)"
IMT,HEALTH IMT
Job Requisition,"R0081541 Data Scientist, Senior (Open)"
Job Requisition Type,Associate
Cluster,Boston Cluster
Time Type,2020-04-01
Job Posting Title,"Advanced Data Scientist/Researcher, Associate"
Safi Recommendation,NaN
Recruiting Start Date,Regular


In [ ]:

from datetime import datetime

# Manually note when the email has been sent
req_id = 'R0071707'.strip()
match_series = (hunting_df['Job Requisition ID'] == req_id)
date_str = '3/16/2020 11:58 AM'
date_format = '%m/%d/%Y %H:%M %p'
date_obj = datetime.strptime(date_str, date_format)
print((datetime.now() - date_obj).days)
hunting_df.loc[match_series, 'opportunity_application_email_date'] = pd.Timestamp(date_obj)
s.store_objects(hunting_df=hunting_df)
last_emailed_dict = s.load_object('last_emailed_dict')
row_series = hunting_df.loc[match_series].iloc[0]
hm_str = row_series['Hiring Manager']
hm_email = clean_email(hm_str)
pr_str = row_series['Primary Recruiter']
pr_email = clean_email(pr_str)
email_tuple = (hm_email, pr_email)
print(email_tuple)
for email in email_tuple:
    if not email in last_emailed_dict:
        last_emailed_dict[email] = pd.Timestamp(date_obj)
s.store_objects(last_emailed_dict=last_emailed_dict)

In [ ]:

print("Here's the managers and recruiters I have spammed in the last two weeks (and the last time I spammed them):")
emails_list = sorted(['{}: {}'.format(key, value.strftime('%B %d')) for key, value in last_emailed_dict.items()])
print('\n'.join(emails_list))

In [ ]:

last_emailed_dict['Lee, David [USA] <544006@bah.com>'] = pd.Timestamp(date_obj)
last_emailed_dict['Nida, Lori [USA] <594683@bah.com>'] = pd.Timestamp(date_obj)
s.store_objects(last_emailed_dict=last_emailed_dict)

In [ ]:

req_id_list = ['R0080204', 'R0081237', 'R0066222', 'R0081079', 'R0067642', 'R0074472', 'R0081024', 'R0080962', 'R0067023', 'R0079762',
               'R0080544', 'R0080214', 'R0080461', 'R0080459', 'R0080211', 'R0072131', 'R0079065', 'R0077973', 'R0079049', 'R0059547', 'R0078739',
               'R0079025', 'R0078760', 'R0078971', 'R0076252', 'R0074969', 'R0074087', 'R0072036', 'R0078549', 'R0078514', 'R0077918', 'R0078696',
               'R0077836', 'R0076794', 'R0072831', 'R0076196', 'R0076889', 'R0076249', 'R0066218', 'R0076197', 'R0076199', 'R0072789']
percent_fit_list = []
for req_id in req_id_list:
    match_series = (hunting_df['Job Requisition ID'] == req_id)
    if hunting_df[match_series].shape[0] == 1:
        percent_fit = hunting_df.loc[match_series, 'percent_fit'].tolist()[0]
        if percent_fit >= 0.8:
            percent_fit_list.append(req_id)
match_series = (hunting_df['Job Requisition ID'].isin(percent_fit_list))
print_emails(match_series)

In [ ]:

print_emails(match_series)

In [ ]:

req_id = 'R0080544'
match_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[match_series, 'manager_notes'].tolist()[0]

In [ ]:

print(hunting_df.groupby('is_opportunity_application_emailed').count().T.max().sort_values(ascending=False))

In [ ]:

print(hunting_df.groupby('is_remote_delivery').count().T.max().sort_values(ascending=False))

In [ ]:

def f(x):
    if ('RAZOR' in x):
        match = True
    else:
        match = False
    
    return match
match_series = hunting_df['Job Description'].map(f)
if hunting_df[match_series].shape[0] > 0:
    print(hunting_df[match_series].shape)
    print(hunting_df[match_series].groupby('Required Clearance').count().T.max().sort_values(ascending=False))
    hunting_df[match_series].head(5).T

In [ ]:

hunting_df.columns.tolist()

In [ ]:

match_series = (hunting_df['is_remote_delivery'] == True) & (hunting_df['is_opportunity_application_emailed'] == True)
columns_list = ['Management Level', 'Job Posting Title', 'Job Profile', 'Job Family Group', 'Primary Location',
                'percent_fit']
df = hunting_df[match_series][columns_list].copy()
df.columns = ['Level', 'Title', 'Profile', 'Group', 'Location', 'Fit']
df.sort_values(['Fit', 'Title'], ascending=[False, True])

In [ ]:

req_id = 'R0079210'
print_job_description(req_id)

In [ ]:

def f(x):
    if re.search(r'\bIAT\b', str(x)):
        results = True
    else:
        results = False
    
    return results

match_series = basic_quals_df.qualification_str.map(f)
for qual in basic_quals_df[match_series].qualification_str.tolist():
    print('•\t{} = {}'.format(qual, basic_quals_dict[qual]))

In [ ]:

basic_quals_dict['Ability to operate independently and manage staff'] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)

In [ ]:

req_id = 'R0079210'
match_series = (hunting_df['Job Requisition ID'] == req_id)
print(hunting_df[match_series]['percent_fit'].tolist())
for row_index, row_series in hunting_df[match_series].iterrows():
    quals_list, job_fitness = print_fit_job(row_index, row_series, basic_quals_dict)

In [ ]:

hunting_df.loc[504, 'percent_fit'] = (1+1+1+1+0+1+1)/7
s.store_objects(hunting_df=hunting_df)

In [ ]:

req_id_list = ['R0073564', 'R0073583', 'R0073584', 'R0073585', 'R0073586']
match_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df[match_series].T

In [ ]:

match_series = (hunting_df.index == 437)
print(hunting_df[match_series]['Job Description'].tolist()[0])

In [ ]:

match_series = (hunting_df.percent_fit >= 0.0)
print(hunting_df[~match_series].sample(1)['Job Description'].tolist()[0])

In [ ]:

#print(['hunting_df.{}'.format(fn) for fn in dir(hunting_df) if 'dup' in fn.lower()])
match_series = hunting_df.duplicated(subset='Job Requisition ID', keep=False)
print(hunting_df[match_series].shape)

In [ ]:

columns_list = ['Hiring Manager', 'Management Level', 'IMT', 'Job Requisition', 'Job Requisition Type', 'Cluster', 'Time Type',
                'Job Posting Title', 'Recruiting Start Date', 'Account Group', 'Job Requisition ID', 'Job Type',
                'Supervisory Organization', 'Clearance Agency', 'Primary Location State/Province', 'Furthest Stage',
                'Resource Manager', 'Primary Location', 'Job Description', 'Group', 'Job Profile', 'Job Family Group', 'FSO',
                'Job Family', 'Job Requisition Status', 'Business Title', 'Job Posting', 'Primary Location Country',
                'Required Clearance', 'Primary Recruiter']
hunting_df = hunting_df.drop_duplicates(subset=columns_list, ignore_index=True)
s.store_objects(hunting_df=hunting_df)

In [ ]:

idx_list = hunting_df[match_series].index.tolist()
first = idx_list[0]
second = idx_list[1]
columns_list = []
for column_name in hunting_df.columns:
    if hunting_df.loc[first, column_name] == hunting_df.loc[second, column_name]:
        columns_list.append(column_name)
columns_list

In [ ]:

match_series = hunting_df.percent_fit.isnull()
print(hunting_df[match_series].shape)
req_id = hunting_df.loc[481, 'Job Requisition ID']
print_job_description(req_id)

In [ ]:

print(['{}'.format(fn) for fn in hunting_df.columns if 'req' in fn.lower()])

In [ ]:

key_regex = re.compile(r'([^0-9A-Za-z\+ \/)(:,]+)-')
for old_key in basic_quals_dict.keys():
    match_obj = key_regex.search(old_key)
    if match_obj:
        print('"{}": {}'.format(match_obj.group(1), old_key))
        #new_key = re.sub('^[?â-]+', '', old_key)
        #print(new_key)
        #basic_quals_dict[new_key] = basic_quals_dict.pop(old_key)
        break

In [ ]:

key_regex = re.compile(r'\s+$')
old_key_list = basic_quals_dict.copy().keys()
for old_key in old_key_list:
    match_obj = key_regex.search(old_key)
    if match_obj:
        #print('"{}": {}'.format(match_obj.group(1), old_key))
        new_key = re.sub(r'\s+$', '', old_key)
        #print(new_key)
        basic_quals_dict[new_key] = basic_quals_dict.pop(old_key)
        #print(old_key)
        #break


---
# Run these to clear out the dataset for the demo

In [ ]:

columns_list = ['Hiring Manager', 'Management Level', 'IMT', 'Job Requisition', 'Job Requisition Type', 'Cluster', 'Time Type',
                'Job Posting Title', 'Safi Recommendation', 'Recruiting Start Date', 'Account Group', 'Job Requisition ID',
                'Job Type', 'Supervisory Organization', 'Clearance Agency', 'Primary Location State/Province', 'Furthest Stage',
                'Resource Manager', 'Primary Location', 'Job Description', 'Group', 'Job Profile', 'Job Family Group', 'FSO',
                'Job Family', 'Job Requisition Status', 'Business Title', 'Job Posting', 'Primary Location Country',
                'Required Clearance', 'Primary Recruiter', 'percent_fit', 'is_opportunity_application_emailed', 'opportunity_application_email_date',
                'is_remote_delivery', 'manager_notes', 'CS Notes']
hunting_df = pd.DataFrame([], columns=columns_list)

In [ ]:

[qual_str for qual_str in basic_quals_dict if 'python' in qual_str.lower()][0]

In [ ]:

old_key = '3 years of experience with developing software in object-oriented and scripting languages, including MATLAB, C/C++, and Python'
if old_key in basic_quals_dict:
    basic_quals_dict.pop(old_key)
req_id_list = ['R0069317']
match_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[match_series, 'percent_fit'] = np.nan


---
# Emailing

In [ ]:

import win32com.client

outlook = win32com.client.Dispatch('Outlook.Application').GetNamespace('MAPI')
print(['outlook.{}'.format(fn) for fn in dir(outlook) if not fn.startswith('_')])


---
# Manually score unscored jobs

In [ ]:

hunting_df.loc[436, 'percent_fit'] = (0+1+1+1+1+1)/6
s.store_objects(hunting_df=hunting_df)


---
# Study of the Safi recommendations

In [ ]:

import random

match_series = (hunting_df['Safi Recommendation'] == 1)
[c[10:100].strip() for c in random.choices(population=hunting_df[match_series]['Job Description'].unique(), k=10)]

In [ ]:

s.store_objects(hunting_df=hunting_df)
match_series = (hunting_df['Safi Recommendation'] == 1)
hunting_df[match_series]['Primary Location State/Province'].unique().tolist()

In [ ]:

hunting_df[match_series]['Job Requisition'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Cluster'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Job Family'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Account Group'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Resource Manager'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Job Requisition Type'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Job Posting'].unique()[:10].tolist()

In [ ]:

item_list = population=hunting_df[match_series]['Hiring Manager'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['IMT'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Required Clearance'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Job Profile'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Management Level'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Clearance Agency'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Group'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Job Profile'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

[cn for cn in columns_list if 'loca' in cn.lower()]

In [ ]:

item_list = population=hunting_df[match_series]['Primary Location'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Primary Recruiter'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])


---
# Initial dataframe creation (don't run again)

In [ ]:

hunting_dir = r'D:\Documents\Administrivia\Job Hunting'
file_name = 'Copy of Reqs Babbitt Applied to_Need Follow Up.3.19.2020.xlsx'
file_path = os.path.join(hunting_dir, file_name)
safi_df = pd.read_excel(file_path)
hunting_df['CS Notes'] = ''
for row_index, row_series in safi_df.iterrows():
    req_id = row_series['Job Requisition ID']
    cs_notes = row_series['CS Notes']
    match_series = (hunting_df['Job Requisition ID'] == req_id)
    hunting_df.loc[match_series, 'CS Notes'] = cs_notes
s.store_objects(hunting_df=hunting_df)

In [ ]:

match_series = (hunting_df.is_opportunity_application_emailed == True) & hunting_df.is_remote_delivery.isnull()
s.save_dataframes(unresponses_df=hunting_df[match_series])

In [ ]:

match_series = hunting_df.manager_notes.isnull()
hunting_df.loc[match_series, 'manager_notes'] = ''
s.store_objects(hunting_df=hunting_df)

In [ ]:

for old_key in basic_quals_dict.keys():
    basic_quals_dict[re.sub(r':$', '', str(old_key))] = basic_quals_dict.pop(old_key)

In [ ]:

match_series = (hunting_df.is_for_university_recruiting == 1)
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

In [ ]:

columns_list = ['Hiring Manager', 'Management Level', 'IMT', 'Job Requisition', 'Job Requisition Type', 'Cluster', 'Time Type',
                'Job Posting Title', 'Safi Recommendation', 'Recruiting Start Date', 'Account Group', 'Job Requisition ID',
                'Job Type', 'Supervisory Organization', 'Clearance Agency', 'Primary Location State/Province', 'Furthest Stage',
                'Resource Manager', 'Primary Location', 'Job Description', 'Group', 'Job Profile', 'Job Family Group', 'FSO',
                'Job Family', 'Job Requisition Status', 'Business Title', 'Job Posting', 'Primary Location Country',
                'Required Clearance', 'Primary Recruiter', 'percent_fit', 'is_opportunity_application_emailed', 'opportunity_application_email_date', 'is_remote_delivery']
hunting_df = hunting_df[columns_list]
s.store_objects(hunting_df=hunting_df)

In [ ]:

hunting_dir = r'D:\Documents\Administrivia\Job Hunting'
columns_list = []
for root, dirs, files in os.walk(hunting_dir):
    #path = root.split(os.sep)
    #print((len(path)-1) * '---', os.path.basename(root))
    for file in files:
        #print(len(path) * '---', file)
        if file.endswith('.csv'):
            print(file)
            file_name = os.path.join(hunting_dir, file)
            if os.path.isfile(file_name):
                df = pd.read_csv(file_name, encoding='iso8859-1')
                columns_list = list(set(columns_list) | set(df.columns.tolist()))

In [ ]:

hunting_df = pd.DataFrame([], columns=columns_list)

for root, dirs, files in os.walk(hunting_dir):
    for file in files:
        if file.endswith('.csv'):
            file_name = os.path.join(hunting_dir, file)
            if os.path.isfile(file_name):
                df = pd.read_csv(file_name, encoding='iso8859-1')
                hunting_df = pd.concat([hunting_df, df])


---

In [ ]:

command_str = '{sys.executable} -m pip install pyOutlook'.format(sys=sys)
print(command_str)
!{command_str}


---
# Flag setting

In [22]:

hunting_df.columns

Index(['Job Posting', 'Job Requisition', 'Job Requisition ID',
       'Job Requisition Status', 'Furthest Stage', 'Supervisory Organization',
       'Group', 'Account Group', 'IMT', 'Cluster', 'Primary Recruiter',
       'Resource Manager', 'Hiring Manager', 'Job Posting Title',
       'Job Profile', 'Job Requisition Type', 'Management Level',
       'Primary Location', 'Primary Location State/Province',
       'Primary Location Country', 'Required Clearance', 'Clearance Agency',
       'Time Type', 'Recruiting Start Date', 'Job Type', 'Job Family',
       'Business Title', 'Job Family Group', 'Job Description',
       'Job Profile Skills', 'percent_fit',
       'is_opportunity_application_emailed',
       'opportunity_application_email_date', 'is_remote_delivery',
       'manager_notes', 'CS Notes'],
      dtype='object')

In [ ]:

column_name = 'Primary Recruiter'
last_name = 'Sallis'.lower()
first_name = 'Michael'.lower()

def f(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

column_name = 'Hiring Manager'
last_name = 'Arunachalam'.lower()
first_name = 'Siva'.lower()

def g(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

match_series = hunting_df[column_name].map(f) & hunting_df[column_name].map(g)
req_id_list = hunting_df[match_series]['Job Requisition ID'].tolist()
formatted_str = 'Thanks for the feedback. I have marked all the reqs you are associated with ({}).'
print(formatted_str.format(conjunctify_list(req_id_list)))
match_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[match_series, 'is_remote_delivery'] = False
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

In [ ]:

# Currently all our roles are local to the DC Metro Area.
req_id = 'R0079210'
match_series = (hunting_df['Job Requisition ID'] == req_id)
location = hunting_df[match_series]['Primary Location'].tolist()[0]
manager = hunting_df[match_series]['Hiring Manager'].tolist()[0]
match_series = (hunting_df['Primary Location'] == location)
print('Is she talking about the {} reqs in her location?'.format(hunting_df[match_series].shape[0]))
match_series = (hunting_df['Hiring Manager'] == manager)
print('Is she talking about the {} reqs she is managing?'.format(hunting_df[match_series].shape[0]))
match_series = (hunting_df['Primary Location'] == location) & (hunting_df['Hiring Manager'] == manager)
formatted_str = 'Thank you for the feedback. I am assuming you are talking about the {} reqs you are managing at your location ({}).'

# Set remote delivery for this list of jobs
req_id_list = hunting_df[match_series]['Job Requisition ID'].tolist()
match_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[match_series, 'is_remote_delivery'] = False
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)
print(formatted_str.format(hunting_df[match_series].shape[0], conjunctify_list(req_id_list)))

In [ ]:

column_name = 'Hiring Manager'
last_name = 'Cofrancesco'.lower()
first_name = 'Elaine'.lower()

def f(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

match_series = hunting_df[column_name].map(f)
req_id_list = hunting_df[match_series]['Job Requisition ID'].tolist()
formatted_str = 'Thanks for the feedback. Does this include all the reqs you are associated with ({})?'
print(formatted_str.format(conjunctify_list(req_id_list)))
match_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[match_series, 'is_remote_delivery'] = False
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

In [289]:

column_name = 'Hiring Manager'
last_name = 'Jendzejec'.lower()
first_name = 'Alexander'.lower()

def f(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

match_series = hunting_df[column_name].map(f)
hunting_df.loc[match_series, 'is_remote_delivery'] = False
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [89]:

column_name = 'Hiring Manager'
last_name = 'ferraro'
first_name = 'mike'

def f(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

match_series = hunting_df[column_name].map(f)
hunting_df.loc[match_series, 'is_remote_delivery'] = False
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [81]:

# Set remote delivery for this list of jobs
req_id_list = ['R0072627', 'R0072630', 'R0072685', 'R0072688', 'R0072699', 'R0072700', 'R0078787', 'R0078792', 'R0078793', 'R0072670',
               'R0072672', 'R0078798', 'R0078799']
match_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[match_series, 'is_remote_delivery'] = False
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [38]:

# Set remote delivery for this list of jobs
req_id_list = ['R0073564', 'R0073583', 'R0073584', 'R0073585', 'R0073586']
match_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[match_series, 'is_remote_delivery'] = True
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [ ]:

# Set university recruiting for this job
req_id = 'R0066388'
match_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[match_series, 'is_for_university_recruiting'] = 1
s.store_objects(hunting_df=hunting_df)

In [ ]:

# Manually note you can't do this job from home
req_id = 'R0064764'
match_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[match_series, 'is_remote_delivery'] = False
s.store_objects(hunting_df=hunting_df)

In [ ]:

# Manually note you can't do this job from home
hunting_df.loc[83, 'is_remote_delivery'] = False
s.store_objects(hunting_df=hunting_df)

In [ ]:

match_series = (hunting_df.index == 2616)
hunting_df.loc[match_series, 'is_for_university_recruiting'] = 1
hunting_df.loc[match_series, 'percent_fit'] = 0.0
s.store_objects(hunting_df=hunting_df)